In [1]:
import boto3
import os
import time
from typing import List, Tuple
from dotenv import load_dotenv
import pandas as pd
from io import StringIO, BytesIO
import io
import tempfile
import json
load_dotenv()

# # env variables
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_SESSION_TOKEN = os.getenv('AWS_SESSION_TOKEN')
REGION_NAME = os.getenv('REGION_NAME')

In [12]:
from botocore.exceptions import ClientError

s3 = boto3.client('s3', region_name=REGION_NAME, aws_access_key_id=AWS_ACCESS_KEY_ID,
                  aws_secret_access_key=AWS_SECRET_ACCESS_KEY, aws_session_token=AWS_SESSION_TOKEN)

bucket_name = "ref-pr-detail-trkw-glbl"
object_key = "QA-data/smart_audits_trkw_audit_fields.xlsx"
data = pd.DataFrame()
file = s3.get_object(Bucket=bucket_name, Key=object_key)
data = file['Body'].read()
df = pd.read_excel(io.BytesIO(data))

df.head(2)

,pr_id,data_field_nm,field_value
0,1318886,Audit Conduct,PurposeA routine investigator site audit of Pr...
1,1318886,Audit Distribution List,Judith K MurphyMelanie IvarssonJenipher Dalton...


In [13]:
df_pivot = df.pivot_table(index='pr_id', columns='data_field_nm', values='field_value', 
                          aggfunc='first').reset_index()
df_pivot.fillna("NaN") 
df_pivot.head(2)

data_field_nm,pr_id,Audit Category,Audit Classification (R&D),Audit Conduct,Audit Conduct Comments,Audit Country,Audit Date - Performed (End),Audit Date - Performed (Start),Audit Distribution List,Audit Execution,...,Study Number,Study Protocols,Submitted for Audit Appr. by,Submitted for Audit Appr. on,Supplier Location,Supplier Locations,Supporting QAC Group,Therapeutic Area,Trackwise Reference Record,Year Planned
0,1318886,Routine,Acceptable,PurposeA routine investigator site audit of Pr...,NaN,NaN,2018-10-24 00:00:00,2018-10-23 00:00:00,Judith K MurphyMelanie IvarssonJenipher Dalton...,On-Site Audit,...,NaN,1317557,"Popiolek, Sherry",2018-11-29 14:03:00,1317774,NaN,Clinical & Medical Compliance,NaN,NaN,2018
1,1318890,Routine,Acceptable,PurposeA routine investigator site audit of Pr...,NaN,NaN,2018-10-26 00:00:00,2018-10-25 00:00:00,Judith K MurphyMelanie IvarssonJenipher Dalton...,On-Site Audit,...,NaN,1317562,"Popiolek, Sherry",2018-11-29 14:47:00,1317774,NaN,Clinical & Medical Compliance,NaN,NaN,2018


In [15]:
df_pivot.fillna("NA",inplace=True)
df_pivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3043 entries, 0 to 3042
Data columns (total 69 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   pr_id                           3043 non-null   int64 
 1   Audit Category                  3043 non-null   object
 2   Audit Classification (R&D)      3043 non-null   object
 3   Audit Conduct                   3043 non-null   object
 4   Audit Conduct Comments          3043 non-null   object
 5   Audit Country                   3043 non-null   object
 6   Audit Date - Performed (End)    3043 non-null   object
 7   Audit Date - Performed (Start)  3043 non-null   object
 8   Audit Distribution List         3043 non-null   object
 9   Audit Execution                 3043 non-null   object
 10  Audit Plan Attachments          3043 non-null   object
 11  Audit Scheduled End Date        3043 non-null   object
 12  Audit Scheduled Start Date      3043 non-null   

In [ ]:
columns_to_chunk = df_pivot.columns

In [30]:
len(columns_to_chunk)

69

In [31]:
new_df = pd.DataFrame()
new_df['combined_text'] = df_pivot[columns_to_chunk].apply(
    lambda row: ' '.join([f"{col}: {row[col]} " for col in columns_to_chunk]), axis=1)


In [32]:
new_df.head(2)

,combined_text
0,pr_id: 1318886 Audit Category: Routine Audit...
1,pr_id: 1318890 Audit Category: Routine Audit...


In [16]:
df_pivot.to_csv("qa_data.csv",index=False)

In [ ]:
# pd.set_option("display.max_colwidth", None)

In [11]:
df_pivot.fillna("NA")


AttributeError: 'NoneType' object has no attribute 'fillna'